# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

print(sum(generator))


8.382332347441762


In [11]:
for res in square_root_generator(13):
    pass
print(res)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# for person in people_1():
#     print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# for person in people_2():
#     print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_generators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
                               table_name="people",
                               write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_generators
The duckdb destination used duckdb:////Users/alexlitvinov/Documents/projects/data-engineering-zoomcamp-alex/dlt-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707681915.562469 is LOADED and contains no failed jobs


In [4]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707681915.562469,OPHzmxpsHL9aVQ
1,2,Person_2,27,City_A,1707681915.562469,S8xogpydnTsDDg
2,3,Person_3,28,City_A,1707681915.562469,ZwsfypLgKZ5KKg
3,4,Person_4,29,City_A,1707681915.562469,jMjDFElB88oeHw
4,5,Person_5,30,City_A,1707681915.562469,3sgk9MJaS1626Q


In [5]:
print(conn.sql("SELECT SUM(age) FROM people").df())

   sum(age)
0     140.0


In [12]:
generators_pipeline.run(people_2(),
                        table_name="people",
                        write_disposition="append")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x12765ff10>, metrics={'1707682384.4805691': [{'started_at': DateTime(2024, 2, 11, 20, 13, 4, 900878, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 11, 20, 13, 5, 85285, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////Users/alexlitvinov/Documents/projects/data-engineering-zoomcamp-alex/dlt-workshop/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_generators', loads_ids=['1707682384.4805691'], load_packages=[LoadPackageInfo(load_id='1707682384.4805691', package_path='/Users/alexlitvinov/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1707682384.4805691', state='loaded', schema=Schema dlt_ipykernel_launcher at 6025903440, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'n

In [13]:
print(conn.sql("SELECT SUM(age) FROM people").df())

   sum(age)
0     353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [17]:
people_merge_pipeline = dlt.pipeline(pipeline_name='people_merge',
                                     destination='duckdb', 
                                     dataset_name='people_merge_generators')

people_merge_pipeline.run(people_1(),
                        table_name="people_merged")

people_merge_pipeline.run(people_2(),
                        table_name="people_merged",
                        write_disposition="merge",
                        primary_key="id")

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x134418e50>, metrics={'1707683321.117677': [{'started_at': DateTime(2024, 2, 11, 20, 28, 41, 506233, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 11, 20, 28, 41, 905211, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////Users/alexlitvinov/Documents/projects/data-engineering-zoomcamp-alex/dlt-workshop/people_merge.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_merge_generators', loads_ids=['1707683321.117677'], load_packages=[LoadPackageInfo(load_id='1707683321.117677', package_path='/Users/alexlitvinov/.dlt/pipelines/people_merge/load/loaded/1707683321.117677', state='loaded', schema=Schema people_merge at 5170503184, schema_update={'people_merged': {'name': 'people_merged', 'columns': {'occupation': {'name': 'occu

In [18]:
conn = duckdb.connect(f"{people_merge_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{people_merge_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_merged       │
└─────────────────────┘

In [19]:
print(conn.sql("SELECT SUM(age) FROM people_merged").df())

   sum(age)
0     266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX